Example notebook...

In [1]:
import sys
sys.path.insert(0, '../..')

import pandas as pd
import numpy as np
import helpers.laxdb as laxdb
LaxDB = laxdb.LaxDB

In [2]:
teams = """
SELECT id, name from ncaa.teams;
"""

teamdata = LaxDB().query(teams)

In [7]:
teamdata.head()

,id,name
0,2,Binghamton
1,50,Villanova
2,54,Vermont
3,202,Canisius
4,55,Sacred Heart


In [3]:
games = """
SELECT * from ncaa.games"""

gamedata = LaxDB().query(games)

In [4]:
game_df = gamedata.copy()

In [5]:
colstokeep = ['id', 'season_id', 'home_team_id', 'away_team_id',
       'game_date', 'ncaa_id_old', 'ncaa_id_new', 'location',
       'home_team_score', 'away_team_score', 'attendance', 'game_state']

game_df2 = pd.DataFrame(game_df, columns=colstokeep)

In [6]:
game_df2['res'] = game_df2['home_team_score'] - game_df2['away_team_score']
game_df2['Home_Win'] = np.where(game_df2['res'] > 0, 1, 0)
game_df2 = game_df2[game_df2['game_state'] == 'Completed']

In [7]:
games_df = teamdata.merge(game_df2, left_on='id', right_on='home_team_id')
games_df.head()

,id_x,name,id_y,season_id,home_team_id,away_team_id,game_date,ncaa_id_old,ncaa_id_new,location,home_team_score,away_team_score,attendance,game_state,res,Home_Win
0,2,Binghamton,8,1,2,19,2015-03-28,3832416.0,563345.0,"Vestal, N.Y.",12.0,13.0,237.0,Completed,-1.0,0
1,2,Binghamton,1,1,2,34,2015-02-22,3796712.0,562160.0,"Vestal, NY",9.0,12.0,203.0,Completed,-3.0,0
2,2,Binghamton,10,1,2,49,2015-04-11,3847018.0,563859.0,"Vestal, N.Y.",11.0,15.0,780.0,Completed,-4.0,0
3,2,Binghamton,6,1,2,24,2015-03-21,3833741.0,563085.0,"Vestal, N.Y.",8.0,9.0,250.0,Completed,-1.0,0
4,2,Binghamton,13,1,2,54,2015-04-25,3860428.0,564461.0,"Vestal, N.Y.",13.0,14.0,280.0,Completed,-1.0,0


In [8]:
grouped = games_df.groupby(['season_id','name'])['Home_Win'].agg(['sum','count'])
homeseason_df = grouped.reset_index()
homeseason_df['Home_win_perc'] = round(homeseason_df['sum'] / homeseason_df['count'],3)
homeseason_df.head()

,season_id,name,sum,count,Home_win_perc
0,1,Air Force,4,7,0.571
1,1,Army West Point,3,6,0.500
2,1,Bellarmine,4,7,0.571
3,1,Binghamton,0,6,0.000
4,1,Boston U.,4,7,0.571


In [9]:
grouped2 = games_df.groupby(['name'])['Home_Win'].agg(['sum','count'])
totalhome_df = grouped2.reset_index()
totalhome_df['Home_win_perc'] = round(totalhome_df['sum'] / totalhome_df['count'],3)
totalhome_df.head()

,name,sum,count,Home_win_perc
0,Adams St.,16,51,0.314
1,Adelphi,73,89,0.820
2,Adrian,32,68,0.471
3,Air Force,42,65,0.646
4,Alabama Huntsville,35,69,0.507


In [29]:
#game_df2.head(20)

In [10]:
LaxDB().close()

In [12]:
#home win perc > 50
home_field_adv = totalhome_df[totalhome_df['Home_win_perc']>.5]
home_field_adv.head()

,name,sum,count,Home_win_perc
1,Adelphi,73,89,0.820
3,Air Force,42,65,0.646
4,Alabama Huntsville,35,69,0.507
6,Albion,65,88,0.739
9,Alfred,49,75,0.653


In [13]:
totalhome_df['home_field_adv'] = np.where(totalhome_df['Home_win_perc']>.5, 1, 0)

In [15]:
hfa = totalhome_df.groupby('home_field_adv').count()
hfa

,name,sum,count,Home_win_perc
home_field_adv,,,,
0,188,188,188,188
1,245,245,245,245
